In [1]:
import pymongo
import nltk
import pandas as pd
import random
import csv

In [2]:
pd.options.display.width = 1000
pd.options.display.column_space = 1000
pd.options.display.expand_frame_repr = False
pd.options.display.max_columns = 4000

<b>Retrieving data from mongodb

In [3]:
client = pymongo.MongoClient()

db = client['tweets']
collection = db['tweets_en']


In [4]:
collection.count()

10932

<b> And processing it and putting in dataframe

In [5]:
indexes = [ '_id', 'favouriteCount', 'retweetCount', 'relevance', 'text' ]
columns = ['id', 'favs', 'rt', 'rel', 'text']

In [6]:
with open('tweets_en.csv', 'w') as csvfile:
    wr = csv.writer(csvfile, delimiter=',')
    wr.writerow(columns)
    for r in collection.find():
        row_values = [r[i] for i in indexes]
        wr.writerow(row_values)


In [7]:
data = pd.read_csv('tweets_en.csv', encoding='utf-8')

<b>Try out with extracting tokens and stuff

In [8]:
def wrap_into_list(data):
    return [data]

def make_tokens( row ):
    sents = nltk.sent_tokenize(row[0])
    words = [nltk.word_tokenize(sent) for sent in sents]
    ps_words = [nltk.pos_tag(w) for w in words]
    ners = list(nltk.ne_chunk_sents(ps_words, binary=True))
    
    total_props = list()
    
    for sent in ners:
        for w in sent:
            try:
                label = w.label()
                t = w[0]
                (name, pos) = t
                total_props.append((name, pos, 'NE'))

            except AttributeError:
                if w[1] == 'NNP' or w[1] == 'NNPS':
                    total_props.append(w)
    
    
    return [total_props]
    


In [9]:
tmp = data.sample(10)

In [10]:
t = pd.DataFrame(tmp['text']).apply( lambda x: make_tokens(x) , axis=1 )

<b>Doing for real

In [11]:
data['tokenized_text'] = pd.DataFrame(data['text']).apply(lambda x: make_tokens(x), axis=1)

In [12]:
# tokenized_text = tokenized_text.rename(index=str, columns={'text': 'tokenized_text'})
recognised_ent = list()
recognised_nouns = list()
for tw in data['tokenized_text'].tolist():
    for ent in tw:
        try:
            ok = ent[2]
            recognised_ent.append(ent[0])
        except IndexError:
            recognised_nouns.append(ent[0])
        

In [13]:
to_filter = ['@', 'IS', 'NOT', 'WAS', '||', '+', 'Are', '|', 'No', 'RT', 'Follow', 'FOLLOW']

recognised_nouns = list(map(lambda x: x.replace(r'[\'‘]', ''), filter(lambda x: x not in to_filter, recognised_nouns)))

In [14]:
len(recognised_ent) + len(recognised_nouns)


23549

In [15]:
total_recognized = recognised_ent + recognised_nouns
len(nltk.unique_list(total_recognized))
unique_recognized = nltk.unique_list(total_recognized)
len(unique_recognized)

9482

In [16]:
def make_feature(s):
    return "Recongnised feature {}".format(s)

def func(x, e):
    if e in x:
        return 1
    else:
        return 0
    

def mapping_with_tokens(ds):
    i = 1
    for e in unique_recognized:
        f = make_feature(e)
        ds.loc[:, f] = pd.DataFrame(ds['text']).apply( lambda x: func(x,e), axis=1 )
        print('>>', f)
        print('>> {}/{}'.format(i, len(unique_recognized)))
        i += 1

In [17]:
cp = data.copy()

In [18]:
mapping_with_tokens(cp)

>> Recongnised feature Barça
>> 1/9482
>> Recongnised feature Nameless
>> 2/9482
>> Recongnised feature Female
>> 3/9482
>> Recongnised feature Did
>> 4/9482
>> Recongnised feature Proud
>> 5/9482
>> Recongnised feature Record
>> 6/9482
>> Recongnised feature Twitch
>> 7/9482
>> Recongnised feature Gurmeetramrahim
>> 8/9482
>> Recongnised feature Vinod
>> 9/9482
>> Recongnised feature VIDEO
>> 10/9482
>> Recongnised feature Skip
>> 11/9482
>> Recongnised feature Harry
>> 12/9482
>> Recongnised feature SPEED
>> 13/9482
>> Recongnised feature RT
>> 14/9482
>> Recongnised feature ADD
>> 15/9482
>> Recongnised feature FUEL
>> 16/9482
>> Recongnised feature CAW
>> 17/9482
>> Recongnised feature CAWs
>> 18/9482
>> Recongnised feature Heh
>> 19/9482
>> Recongnised feature Donald
>> 20/9482
>> Recongnised feature Deepest
>> 21/9482
>> Recongnised feature Martyrs
>> 22/9482
>> Recongnised feature Actual
>> 23/9482
>> Recongnised feature Picture
>> 24/9482
>> Recongnised feature Embark
>> 25/948

>> Recongnised feature WEAK
>> 201/9482
>> Recongnised feature God
>> 202/9482
>> Recongnised feature LateLateStyles
>> 203/9482
>> Recongnised feature Naomi
>> 204/9482
>> Recongnised feature Betsey
>> 205/9482
>> Recongnised feature PureLFC
>> 206/9482
>> Recongnised feature Liver
>> 207/9482
>> Recongnised feature NBFootball
>> 208/9482
>> Recongnised feature FUNNY
>> 209/9482
>> Recongnised feature PARTIES
>> 210/9482
>> Recongnised feature ELECTED
>> 211/9482
>> Recongnised feature PATRIOTIC
>> 212/9482
>> Recongnised feature TRUMP
>> 213/9482
>> Recongnised feature Joe
>> 214/9482
>> Recongnised feature Chorong
>> 215/9482
>> Recongnised feature Fancafe
>> 216/9482
>> Recongnised feature Apple
>> 217/9482
>> Recongnised feature Honor
>> 218/9482
>> Recongnised feature Event
>> 219/9482
>> Recongnised feature OMGGGGG
>> 220/9482
>> Recongnised feature PANN
>> 221/9482
>> Recongnised feature OriginalFunko
>> 222/9482
>> Recongnised feature Aliens
>> 223/9482
>> Recongnised feature 

>> Recongnised feature Trend
>> 397/9482
>> Recongnised feature BUSH
>> 398/9482
>> Recongnised feature LOCAL
>> 399/9482
>> Recongnised feature DAMN
>> 400/9482
>> Recongnised feature OH
>> 401/9482
>> Recongnised feature Jungkook
>> 402/9482
>> Recongnised feature Haphazard
>> 403/9482
>> Recongnised feature Congress
>> 404/9482
>> Recongnised feature BJP
>> 405/9482
>> Recongnised feature Absolute
>> 406/9482
>> Recongnised feature Bitter
>> 407/9482
>> Recongnised feature Ashamed
>> 408/9482
>> Recongnised feature Aghast
>> 409/9482
>> Recongnised feature Tomorrow
>> 410/9482
>> Recongnised feature OneForKiefLy
>> 411/9482
>> Recongnised feature AFTER
>> 412/9482
>> Recongnised feature JONGDAES
>> 413/9482
>> Recongnised feature THAT
>> 414/9482
>> Recongnised feature Kourtney
>> 415/9482
>> Recongnised feature RAZER
>> 416/9482
>> Recongnised feature Via
>> 417/9482
>> Recongnised feature Kushner
>> 418/9482
>> Recongnised feature Flynn
>> 419/9482
>> Recongnised feature Manafort


>> Recongnised feature Diba
>> 592/9482
>> Recongnised feature Simply
>> 593/9482
>> Recongnised feature PayPal
>> 594/9482
>> Recongnised feature Classic
>> 595/9482
>> Recongnised feature Mother
>> 596/9482
>> Recongnised feature Want
>> 597/9482
>> Recongnised feature Shake
>> 598/9482
>> Recongnised feature CNBLUE
>> 599/9482
>> Recongnised feature Cesare
>> 600/9482
>> Recongnised feature Mario
>> 601/9482
>> Recongnised feature Wembley
>> 602/9482
>> Recongnised feature Season
>> 603/9482
>> Recongnised feature LineofDuty
>> 604/9482
>> Recongnised feature Tiffany
>> 605/9482
>> Recongnised feature Yoona
>> 606/9482
>> Recongnised feature Cotty
>> 607/9482
>> Recongnised feature Kai
>> 608/9482
>> Recongnised feature Mr.
>> 609/9482
>> Recongnised feature Yep
>> 610/9482
>> Recongnised feature JFK
>> 611/9482
>> Recongnised feature Okkodo
>> 612/9482
>> Recongnised feature Sooyoung
>> 613/9482
>> Recongnised feature Joon
>> 614/9482
>> Recongnised feature Bok
>> 615/9482
>> Recon

>> Recongnised feature Pep
>> 787/9482
>> Recongnised feature Michael
>> 788/9482
>> Recongnised feature MSG
>> 789/9482
>> Recongnised feature Bonsai
>> 790/9482
>> Recongnised feature CatchaRUSSpy
>> 791/9482
>> Recongnised feature Fought
>> 792/9482
>> Recongnised feature ARSLEI
>> 793/9482
>> Recongnised feature NFL
>> 794/9482
>> Recongnised feature Medical
>> 795/9482
>> Recongnised feature Afghan
>> 796/9482
>> Recongnised feature SMEnt
>> 797/9482
>> Recongnised feature Seohyun
>> 798/9482
>> Recongnised feature Steak
>> 799/9482
>> Recongnised feature BGC
>> 800/9482
>> Recongnised feature Taehyung
>> 801/9482
>> Recongnised feature HO
>> 802/9482
>> Recongnised feature HOURS
>> 803/9482
>> Recongnised feature SAAHOTEASER
>> 804/9482
>> Recongnised feature Sweden
>> 805/9482
>> Recongnised feature Coulter
>> 806/9482
>> Recongnised feature BTS_twt
>> 807/9482
>> Recongnised feature ARMY
>> 808/9482
>> Recongnised feature LOVE
>> 809/9482
>> Recongnised feature ALWAYS
>> 810/94

...

>> Recongnised feature AubreyAllen
>> 6272/9482
>> Recongnised feature Top…
>> 6273/9482
>> Recongnised feature Imma
>> 6274/9482
>> Recongnised feature SportsCenter
>> 6275/9482
>> Recongnised feature Snack
>> 6276/9482
>> Recongnised feature Meal
>> 6277/9482
>> Recongnised feature Heartbreaking
>> 6278/9482
>> Recongnised feature Airport
>> 6279/9482
>> Recongnised feature Rights
>> 6280/9482
>> Recongnised feature Commission
>> 6281/9482
>> Recongnised feature MissThisKiss
>> 6282/9482
>> Recongnised feature Yesung
>> 6283/9482
>> Recongnised feature SpringFalling
>> 6284/9482
>> Recongnised feature ShawnMendes
>> 6285/9482
>> Recongnised feature ❤️❤️❤️
>> 6286/9482
>> Recongnised feature Stab
>> 6287/9482
>> Recongnised feature 💔
>> 6288/9482
>> Recongnised feature 😇🤗😇
>> 6289/9482
>> Recongnised feature IFTRCG
>> 6290/9482
>> Recongnised feature Hug
>> 6291/9482
>> Recongnised feature Cuddle
>> 6292/9482
>> Recongnised feature 👼🏼💙👼🏼💙
>> 6293/9482
>> Recongnised feature Informatio

>> Recongnised feature 😂😍
>> 6459/9482
>> Recongnised feature 🚨
>> 6460/9482
>> Recongnised feature Caves
>> 6461/9482
>> Recongnised feature Group👉🏻CAIR
>> 6462/9482
>> Recongnised feature RADICAL
>> 6463/9482
>> Recongnised feature Cleric
>> 6464/9482
>> Recongnised feature Review
>> 6465/9482
>> Recongnised feature Anti-Terror
>> 6466/9482
>> Recongnised feature Progra…
>> 6467/9482
>> Recongnised feature Chat
>> 6468/9482
>> Recongnised feature AlDubBigBoyz…
>> 6469/9482
>> Recongnised feature NovaNation
>> 6470/9482
>> Recongnised feature PER🛫
>> 6471/9482
>> Recongnised feature 갓세븐
>> 6472/9482
>> Recongnised feature 🤷🏾‍♂️
>> 6473/9482
>> Recongnised feature SmallBusiness…
>> 6474/9482
>> Recongnised feature Dry
>> 6475/9482
>> Recongnised feature Shampoo
>> 6476/9482
>> Recongnised feature Sure
>> 6477/9482
>> Recongnised feature TheWaffleBus
>> 6478/9482
>> Recongnised feature BIEBER
>> 6479/9482
>> Recongnised feature DOES
>> 6480/9482
>> Recongnised feature SERVE
>> 6481/9482

>> Recongnised feature ABA
>> 6647/9482
>> Recongnised feature Nigel_Farage
>> 6648/9482
>> Recongnised feature B747-400
>> 6649/9482
>> Recongnised feature FAN
>> 6650/9482
>> Recongnised feature Se…
>> 6651/9482
>> Recongnised feature Justice
>> 6652/9482
>> Recongnised feature Roberts
>> 6653/9482
>> Recongnised feature Extraordinary
>> 6654/9482
>> Recongnised feature ≧v≦
>> 6655/9482
>> Recongnised feature ArijitSingh
>> 6656/9482
>> Recongnised feature ChannaMereya
>> 6657/9482
>> Recongnised feature THROW
>> 6658/9482
>> Recongnised feature 🔪🍚
>> 6659/9482
>> Recongnised feature Sexist
>> 6660/9482
>> Recongnised feature CC
>> 6661/9482
>> Recongnised feature TDJakesShow…
>> 6662/9482
>> Recongnised feature IDEA
>> 6663/9482
>> Recongnised feature THINGS
>> 6664/9482
>> Recongnised feature PORTAL
>> 6665/9482
>> Recongnised feature WERE
>> 6666/9482
>> Recongnised feature GRAFTED
>> 6667/9482
>> Recongnised feature BONE
>> 6668/9482
>> Recongnised feature Fortune..
>> 6669/9482


>> Recongnised feature DrBeckmannUK
>> 6831/9482
>> Recongnised feature Colour
>> 6832/9482
>> Recongnised feature Laugh
>> 6833/9482
>> Recongnised feature MS-13
>> 6834/9482
>> Recongnised feature SabrinaDeTour
>> 6835/9482
>> Recongnised feature Direction
>> 6836/9482
>> Recongnised feature WKO_tgirl_lover
>> 6837/9482
>> Recongnised feature TvFanSlw
>> 6838/9482
>> Recongnised feature SheCockLove…
>> 6839/9482
>> Recongnised feature LIST
>> 6840/9482
>> Recongnised feature PLAYFUL
>> 6841/9482
>> Recongnised feature AFFECTIONATE
>> 6842/9482
>> Recongnised feature CAT/DOG/KID
>> 6843/9482
>> Recongnised feature HELPDOGS
>> 6844/9482
>> Recongnised feature URGENTPODR.ORG
>> 6845/9482
>> Recongnised feature Air
>> 6846/9482
>> Recongnised feature Force
>> 6847/9482
>> Recongnised feature 📌
>> 6848/9482
>> Recongnised feature Cr.Vivid
>> 6849/9482
>> Recongnised feature justinbieber
>> 6850/9482
>> Recongnised feature LuisFonsi
>> 6851/9482
>> Recongnised feature LEVEL
>> 6852/9482
>>

KeyboardInterrupt: 

In [19]:
cp.shape

(10932, 7019)

In [20]:
cp.to_csv('features_seven.csv', encoding='utf-8')